In [5]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:0000:0100:03
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 4.0 MB/s eta 0:00:00a 0:00:01
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=5ec5562e356ae2e9d198b9eee8ea796d33123b1dd71603c1e8009b2220e5e8e5
  Stored in directory: /Users/ibrahim/Library/Caches/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark


In [1]:
from warnings import filterwarnings
filterwarnings(action='ignore')
import findspark
findspark.init('/Users/ibrahim/spark/spark-3.5.1-bin-hadoop3')

In [2]:
import pyspark
from pyspark import SparkContext

## Konfigürasyon ve Spark Bağlantısı

In [3]:
from pyspark import SparkContext
sc = SparkContext(master="local")
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/17 22:45:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=pyspark-shell>

24/05/17 22:45:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


## Spark Versiyonu için

In [4]:
sc.version

'3.5.1'

## Kullanıcı Sorgusu için

In [5]:
sc.sparkUser()

'ibrahim'

In [6]:
dir(sc)

['PACKAGE_EXTENSIONS',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_assert_on_driver',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_encryption_enabled',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_repr_html_',
 '_serialize_to_jvm',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addArchive',
 'addFile',
 'addJobTag',


## Açılan Session'ı kapatmak istersek

In [8]:
sc.stop()
#Yukarıdaki ile kapatırız.


In [10]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.conf import  SparkConf
from pyspark import SparkContext

sc = SparkContext(master = "local")
sc

<SparkContext master=local appName=pyspark-shell>

In [23]:
sc.stop() #sparkı durdurmadan yeni bir spark oluşturamazsınız

## Normal şartlarda yukarıdaki gibi Spark yapısını oluşturabiliriz fakat biz konfigüre edebilmek istersek aşağıdaki gibi yazmamız gerekir.

In [25]:
spark = (SparkSession.builder  
            .master("local") 
            .appName("pyspark for practice") 
            .getOrCreate()
)
sc = spark.sparkContext
sc
# configin normalde boş bırakılması sisteme uygun bir şekilde configlenir. istersek ram ayarları gibi ayarları yapabiliriz.

<SparkContext master=local appName=pyspark for practice>

## Temel Data Frame İşlemleri

### Alışageldiğimiz pandas data frame'leri spark tarafında uyumsuzluğa sebep olabiliyor. Bu açıdan bundan sonra spark data frameleri üzerine çalışacağız.

In [27]:
spark_df = spark.read.csv('./diabetes.csv', header=True, inferSchema=True)

## Şema Görüntülemek İstersek

In [29]:
spark_df.printSchema()

root
 |-- Pregnancies: integer (nullable = true)
 |-- Glucose: integer (nullable = true)
 |-- BloodPressure: integer (nullable = true)
 |-- SkinThickness: integer (nullable = true)
 |-- Insulin: integer (nullable = true)
 |-- BMI: double (nullable = true)
 |-- DiabetesPedigreeFunction: double (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Outcome: integer (nullable = true)



In [30]:
type(spark)

pyspark.sql.session.SparkSession

In [31]:
type(spark_df)

pyspark.sql.dataframe.DataFrame

## Veriyi RAM'e taşımak istersek

In [35]:
spark_df.cache() #veriyi işlenebilir bir hale getirmek olarak da düşünebiliriz

24/05/17 23:29:21 WARN CacheManager: Asked to cache already cached data.


DataFrame[Pregnancies: int, Glucose: int, BloodPressure: int, SkinThickness: int, Insulin: int, BMI: double, DiabetesPedigreeFunction: double, Age: int, Outcome: int]

In [36]:
type(spark_df)

pyspark.sql.dataframe.DataFrame

## Karşılaştırmak Üzere Python DataFrame oluşturalım

In [37]:
import seaborn as sns
df = sns.load_dataset("diamonds")
df = df.select_dtypes(include=["float64","int64"])

In [38]:
type(df)

pandas.core.frame.DataFrame

## Yukarıda da görüldüğü üzere pandas üzerinden bir data frame incelediğimizde pandas core üzerinde bir data frame olduğunu görebiliyoruz.

## İki tarafta da desteklenen bazı fonksiyonları incelemek istersek

In [39]:
df.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [40]:
spark_df.head()

Row(Pregnancies=6, Glucose=148, BloodPressure=72, SkinThickness=35, Insulin=0, BMI=33.6, DiabetesPedigreeFunction=0.627, Age=50, Outcome=1)

In [42]:
df.dtypes

carat    float64
depth    float64
table    float64
price      int64
x        float64
y        float64
z        float64
dtype: object

In [43]:
spark_df.dtypes

[('Pregnancies', 'int'),
 ('Glucose', 'int'),
 ('BloodPressure', 'int'),
 ('SkinThickness', 'int'),
 ('Insulin', 'int'),
 ('BMI', 'double'),
 ('DiabetesPedigreeFunction', 'double'),
 ('Age', 'int'),
 ('Outcome', 'int')]

In [44]:
df.ndim

2

In [45]:
spark_df.ndim

AttributeError: 'DataFrame' object has no attribute 'ndim'

## Görüldüğü üzere numpy üzerinden boyut bildirgesi almak için kullandığımız ndim fonksiyonunu spark'da kullanamıyoruz.

## Aynı durum size için de geçerli 

In [46]:
spark_df.size

AttributeError: 'DataFrame' object has no attribute 'size'